<a href="https://colab.research.google.com/github/yeonghun00/stock_public/blob/main/daytrading_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import files
uploaded = files.upload()

Saving 1 3910 매매내역_(돌파고)_2021-07-31.xls to 1 3910 매매내역_(돌파고)_2021-07-31.xls


In [64]:
import pandas as pd

code_names_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0]
df = pd.read_html(uploaded[list(uploaded)[0]])[0]

In [39]:
import datetime

df.columns = df.iloc[0]
df = df.iloc[1:]

df['거래일자'] = '2021/' + df['거래일자']
df['거래일자'] = list(map(lambda x: datetime.datetime.strptime(x, '%Y/%m/%d %H:%M'), df['거래일자']))
df['매수/매도가'], df['수량'], df['매매금액'] = df['매수/매도가'].astype(float), df['수량'].astype(int), df['매매금액'].astype(int)

In [40]:
df = df[::-1].reset_index(drop=True)
df

,거래일자,매매구분,종목명,매수/매도가,수량,매매금액
0,2021-06-14 09:00:00,매수,이루온,3075.0,726,2232450
1,2021-06-14 09:00:00,매수,이루온,3090.0,319,985710
2,2021-06-14 09:01:00,매도,이루온,3135.0,1045,3276075
3,2021-06-14 09:01:00,매수,이루온,3160.0,1065,3365400
4,2021-06-14 09:01:00,매도,이루온,3180.0,715,2273700
...,...,...,...,...,...,...
12164,2021-07-29 14:27:00,매도,국전약품,8450.0,425,3591250
12165,2021-07-29 14:27:00,매도,국전약품,8450.0,1006,8500700
12166,2021-07-29 14:27:00,매도,국전약품,8430.0,352,2967360
12167,2021-07-29 15:19:00,매수,큐라클,36700.0,409,15010300


In [41]:
code_dic = {}

for i in range(len(df)):
  if df.loc[i]['종목명'] not in list(code_dic):
    code_dic[df.loc[i]['종목명']] = {'time':[df.loc[i]['거래일자']], 'type':[df.loc[i]['매매구분']], 'price':[df.loc[i]['매수/매도가']], 'quant':[df.loc[i]['수량']]}
  else:
    code_dic[df.loc[i]['종목명']]['time'].append(df.loc[i]['거래일자'])
    code_dic[df.loc[i]['종목명']]['type'].append(df.loc[i]['매매구분'])
    code_dic[df.loc[i]['종목명']]['price'].append(df.loc[i]['매수/매도가'])
    code_dic[df.loc[i]['종목명']]['quant'].append(df.loc[i]['수량'])

code_df = pd.DataFrame.from_dict(code_dic).T

In [42]:
code_df

,time,type,price,quant
이루온,"[2021-06-14 09:00:00, 2021-06-14 09:00:00, 202...","[매수, 매수, 매도, 매수, 매도, 매도, 매수, 매도, 매수, 매도, 매수, 매...","[3075.0, 3090.0, 3135.0, 3160.0, 3180.0, 3180....","[726, 319, 1045, 1065, 715, 350, 601, 601, 953..."
디지틀조선,"[2021-06-14 09:01:00, 2021-06-14 09:02:00, 202...","[매수, 매도, 매수, 매도, 매수, 매도, 매수, 매도, 매수, 매도, 매도, 매...","[4305.0, 4295.0, 4380.0, 4375.0, 4365.0, 4540....","[788, 788, 759, 759, 736, 736, 729, 729, 769, ..."
평화홀딩스,"[2021-06-14 09:02:00, 2021-06-14 09:04:00]","[매수, 매도]","[5700.0, 5810.0]","[175, 175]"
한창제지,"[2021-06-14 09:02:00, 2021-06-14 09:02:00, 202...","[매수, 매도, 매수, 매도, 매수, 매도, 매수, 매도, 매수, 매도, 매도, 매...","[3015.0, 3010.0, 3100.0, 3090.0, 3075.0, 3065....","[479, 479, 497, 497, 1126, 1126, 1294, 1294, 3..."
화승코퍼레이션,"[2021-06-14 09:03:00, 2021-06-14 09:06:00, 202...","[매수, 매도, 매수, 매도, 매수, 매도, 매수, 매도, 매수, 매수, 매도, 매...","[2620.0, 2635.0, 2820.0, 2970.0, 2855.0, 2885....","[544, 544, 1228, 1228, 1321, 1321, 2105, 2105,..."
...,...,...,...,...
엘비루셈,"[2021-07-29 09:17:00, 2021-07-29 09:18:00, 202...","[매수, 매도, 매수, 매도, 매도, 매수, 매도, 매도, 매도, 매도]","[16000.0, 16400.0, 16500.0, 16350.0, 16350.0, ...","[937, 937, 914, 63, 851, 914, 9, 568, 157, 180]"
현대비앤지스틸,"[2021-07-29 09:26:00, 2021-07-29 09:26:00, 202...","[매수, 매수, 매수, 매수, 매수, 매수, 매수, 매수, 매수, 매도, 매도, 매...","[23000.0, 23100.0, 23100.0, 23100.0, 23100.0, ...","[196, 1, 30, 5, 11, 10, 86, 98, 215, 100, 19, ..."
KT서브마린,"[2021-07-29 09:33:00, 2021-07-29 09:33:00, 202...","[매수, 매수, 매도, 매수, 매도]","[8030.0, 8030.0, 7993.0, 8100.0, 7990.0]","[903, 639, 1542, 1856, 1856]"
TYM,"[2021-07-29 10:49:00, 2021-07-29 10:49:00, 202...","[매수, 매수, 매수, 매수, 매도, 매도, 매도, 매도, 매도, 매도]","[2175.0, 2185.0, 2185.0, 2185.0, 2190.0, 2190....","[4108, 501, 87, 2232, 5, 1506, 305, 3470, 1367..."


In [43]:
def get_profit(code):
  q = 0
  profit = 0
  av_buy = 0
  av_sell = 0
  for t in [x for x in list(zip(code_df.loc[code]['type'], code_df.loc[code]['price'], code_df.loc[code]['quant']))]:
    if t[0] == '매수':
      q += t[2]
      profit -= t[1] * t[2]
      av_buy += t[1]
    else:
      q -= t[2]
      profit += t[1] * t[2]
      av_sell += t[1]
  av_buy /= code_df.loc[code]['type'].count('매수')
  av_sell /= code_df.loc[code]['type'].count('매도')

  if q != 0: return 0, 1
  return profit, av_sell/av_buy

In [44]:
codes = {}
total_profit = 0
total_per = 1

for code in code_df.index:
  total_profit += get_profit(code)[0]
  total_per *= get_profit(code)[1]
  codes[code] = [get_profit(code)[0], get_profit(code)[1]]

In [45]:
pd.DataFrame.from_dict(codes, orient='index', columns=['profit', 'per']).sort_values(by = 'per', ascending=False).head(10)

,profit,per
액션스퀘어,1474960.0,1.153731
인트로메딕,-435677.0,1.121414
경남제약,1220340.0,1.110903
에코프로,852013.0,1.076614
디지틀조선,302050.0,1.071224
한국정보공학,432080.0,1.059060
지나인제약,-277925.0,1.058458
아이비김영,62932.0,1.054525
대원미디어,402050.0,1.052632
캐리소프트,57475.0,1.045696


In [93]:
def get_code(code):
  return pd.DataFrame.from_dict(code_df.loc[code].to_dict())

def name_to_code(code):
  return str(code_names_df[code_names_df['회사명'] == code]['종목코드'].values[0]).rjust(6, '0')

In [47]:
get_code('광진윈텍')

,time,type,price,quant
0,2021-07-16 09:02:00,매수,5730.0,1208
1,2021-07-16 09:06:00,매도,6170.0,131
2,2021-07-16 09:06:00,매도,6120.0,147
3,2021-07-16 09:06:00,매도,6050.0,930
4,2021-07-16 09:22:00,매수,6080.0,1275
5,2021-07-16 09:23:00,매도,6030.0,865
6,2021-07-16 09:23:00,매도,6030.0,165
7,2021-07-16 09:23:00,매도,6030.0,7
8,2021-07-16 09:23:00,매도,6010.0,1
9,2021-07-16 09:24:00,매도,6010.0,39


In [53]:
!pip install finance-datareader
import FinanceDataReader as fdr

In [97]:
set([x.date() for x in get_code('광진윈텍')['time']])

{datetime.date(2021, 7, 16),
 datetime.date(2021, 7, 19),
 datetime.date(2021, 7, 23)}

In [98]:
def show_traded(code):
  traded_dates = set([x.date() for x in get_code(code)['time']])

  df = fdr.DataReader(name_to_code(code), '2021-06-10', '2021-07-30')

  fdr.chart.plot(df, title=code)

In [99]:
show_traded('광진윈텍')